In [ ]:
!git clone https://github.com/thuongle2210/VietNamProhibitionSign

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
import tensorflow as tf

from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import accuracy_score

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
# loading dataset
data = []
labels = []
cur_path = './VietNamProhibitionSign/VietNamProhibitionSign'
print(os.listdir(cur_path))
for i in os.listdir(cur_path):
    if i != "resize.py":
        dir = cur_path + '/' + i
        count = 0
        for j in os.listdir(dir):
            count+=1
            if count >= 590:
                break
            img_path = dir+'/'+j
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224,224), interpolation = cv2.INTER_NEAREST)
            data.append(img)
            labels.append(i)
        print(i)
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size= 0.2, random_state=21)

print((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

# converting the labels into one hot encoding
y_train = to_categorical(y_train, num_classes= 8)
y_test = to_categorical(y_test, num_classes = 8)


In [ ]:
# number of images in each class
data_dic = {}
for folder in os.listdir(cur_path):
    if folder != "resize.py":
        data_dic[folder] = len(os.listdir(cur_path + '/' + folder))

data_df= pd.Series(data_dic)
plt.figure(figsize = (15, 6))
data_df.sort_values().plot(kind = 'bar')
plt.xlabel('Classes')
plt.ylabel('Number of images')

In [ ]:
# defining model structure
#Lenet 
'''
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', input_shape = (28,28,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(8, activation= 'softmax'))

# model compilation
model.compile(loss = 'categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

'''
model = Sequential()
model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv1',
                 input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv1'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv2'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv1'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv2'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv1'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv2'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(8, activation='softmax'))
def optimizer_init_fn(): 
    learning_rate = 1e-4
    return tf.keras.optimizers.Adam(learning_rate) 
model.compile(optimizer=optimizer_init_fn(),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
!pip install visualkeras
import visualkeras

In [ ]:
visualkeras.layered_view(model)

In [ ]:
epochs = 10
history = model.fit(X_train, y_train, 
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(X_test, y_test))

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label= 'train accuracy')
plt.plot(history.history['val_accuracy'], label= 'test accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
plt.figure(0)
plt.plot(history.history['loss'], label= 'train loss')
plt.plot(history.history['val_loss'], label= 'test loss')
plt.title('Loss')
plt.xlabel('epochs') 
plt.ylabel('Loss')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt


# Định nghĩa biển báo theo lớp
0: Biển báo dừng lại

1: Biển báo cấm người đi bộ

2: Tốc độ tối đa là 50 km/h

3: Cấm quay đầu xe

4: Cấm rẽ phải

5: Cấm rẽ trái

6: Cấm đi ngược chiều

7: Biển đường cấm


In [ ]:
x = 209
print(model.predict(X_test[x:x+1]))
print(model.predict_classes(X_test[x:x+1]))


In [ ]:
plt.imshow(X_test[x])

In [ ]:
x =234
print("Probability of 8 classes")
print(model.predict(X_test[x:x+1]))
print("predict:",model.predict_classes(X_test[x:x+1]))
print("Actual label:", y_test[x])
plt.axis("off")
plt.imshow(cv2.cvtColor(X_test[x], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
x = 258
print(model.predict(X_test[x:x+1]))
print(model.predict_classes(X_test[x:x+1]))


In [ ]:
plt.imshow(X_test[x])

In [ ]:
x = 250
print(model.predict(X_test[x:x+1]))
print(model.predict_classes(X_test[x:x+1]))


In [ ]:
plt.imshow(X_test[x])